In [1]:
import pandas as pd

In [3]:
pd.__version__

'1.4.2'

In [7]:
df = pd.read_parquet("green_tripdata_2023-01.parquet")

In [8]:
df = df.head(100)

In [9]:
df

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
0,2,2023-01-01 00:26:10,2023-01-01 00:37:11,N,1.0,166,143,1.0,2.58,14.9,1.0,0.5,4.03,0.00,None,1.0,24.18,1.0,1.0,2.75
1,2,2023-01-01 00:51:03,2023-01-01 00:57:49,N,1.0,24,43,1.0,1.81,10.7,1.0,0.5,2.64,0.00,None,1.0,15.84,1.0,1.0,0.00
2,2,2023-01-01 00:35:12,2023-01-01 00:41:32,N,1.0,223,179,1.0,0.00,7.2,1.0,0.5,1.94,0.00,None,1.0,11.64,1.0,1.0,0.00
3,1,2023-01-01 00:13:14,2023-01-01 00:19:03,N,1.0,41,238,1.0,1.30,6.5,0.5,1.5,1.70,0.00,None,1.0,10.20,1.0,1.0,0.00
4,1,2023-01-01 00:33:04,2023-01-01 00:39:02,N,1.0,41,74,1.0,1.10,6.0,0.5,1.5,0.00,0.00,None,1.0,8.00,1.0,1.0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2,2023-01-01 01:19:54,2023-01-01 01:23:25,N,1.0,7,179,1.0,0.47,5.8,1.0,0.5,2.49,0.00,None,1.0,10.79,1.0,1.0,0.00
96,2,2023-01-01 01:31:10,2023-01-01 01:37:17,N,1.0,7,179,1.0,1.16,7.9,1.0,0.5,2.08,0.00,None,1.0,12.48,1.0,1.0,0.00
97,2,2023-01-01 01:42:30,2023-01-01 02:26:39,N,1.0,179,226,1.0,15.39,66.0,1.0,0.5,0.00,0.00,None,1.0,68.50,1.0,1.0,0.00
98,2,2023-01-01 00:57:51,2023-01-01 01:07:05,N,5.0,260,74,2.0,5.34,52.0,0.0,0.0,0.00,6.55,None,1.0,59.55,1.0,2.0,0.00
